<a href="https://colab.research.google.com/github/01star01ek/01star01ek/blob/main/deepfake_small_V3_gen_final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# install dependency

In [ ]:
!pip install mediapipe==0.10.11
!pip install opencv-contrib-python flatbuffers==23.5.26 sounddevice==0.4.6 attrs==23.1.0
!pip install torch==2.1.0 torchvision==0.16.0
!pip install dlib opencv-python scikit-image pillow matplotlib imageio gdown tqdm
!pip install ninja tensorboard tensorboardX pyaml pyrallis ftfy
!pip install face-alignment==1.3.5

In [ ]:
!pip install numpy==1.26.4 --force-reinstall

  Using cached numpy-1.26.4-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (61 kB)
Using cached numpy-1.26.4-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (18.3 MB)
  Attempting uninstall: numpy
    Found existing installation: numpy 1.26.4
    Uninstalling numpy-1.26.4:
      Successfully uninstalled numpy-1.26.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
ydf 0.12.0 requires protobuf<6.0.0,>=5.29.1, but you have protobuf 3.20.3 which is incompatible.
tensorflow 2.18.0 requires flatbuffers>=24.3.25, but you have flatbuffers 23.5.26 which is incompatible.
thinc 8.3.6 requires numpy<3.0.0,>=2.0.0, but you have numpy 1.26.4 which is incompatible.


In [ ]:
!mkdir -p /content/extract_frames
!mkdir -p /content/input_videos
!mkdir -p /content/alignmented_frame
!mkdir -p /content/alignmented_frame_aligned
!mkdir -p /content/alignmented_frame_croped
!mkdir -p /content/alignmented_frame_transforms

#git hub & model install

In [ ]:
!git clone https://github.com/yuval-alaluf/stylegan3-editing.git

fatal: destination path 'stylegan3-editing' already exists and is not an empty directory.


## models

In [ ]:
!wget http://dlib.net/files/shape_predictor_68_face_landmarks.dat.bz2 -P /content/pretrained_models/
!bzip2 -d /content/pretrained_models/shape_predictor_68_face_landmarks.dat.bz2

--2025-06-20 12:40:07--  http://dlib.net/files/shape_predictor_68_face_landmarks.dat.bz2
Resolving dlib.net (dlib.net)... 107.180.26.78
Connecting to dlib.net (dlib.net)|107.180.26.78|:80... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://dlib.net/files/shape_predictor_68_face_landmarks.dat.bz2 [following]
--2025-06-20 12:40:07--  https://dlib.net/files/shape_predictor_68_face_landmarks.dat.bz2
Connecting to dlib.net (dlib.net)|107.180.26.78|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 64040097 (61M)
Saving to: ‘/content/pretrained_models/shape_predictor_68_face_landmarks.dat.bz2.1’

shape_predictor_68_ 100%[===================>]  61.07M  43.6MB/s    in 1.4s    

2025-06-20 12:40:09 (43.6 MB/s) - ‘/content/pretrained_models/shape_predictor_68_face_landmarks.dat.bz2.1’ saved [64040097/64040097]

bzip2: Output file /content/pretrained_models/shape_predictor_68_face_landmarks.dat already exists.


In [ ]:
!gdown --id 1z_cB187QOc6aqVBdLvYvBjoc93-_EuRm -O /content/pretrained_models/restyle_e4e_sg3.pt
!gdown --id 13q6m-bpe3Ws9en9y45JEx2PHQirStt8N -O /content/pretrained_models/stylegan3-ffhq-1024x1024.pt
!gdown --id 1KW7bjndL3QG3sxBbZxreGHigcCCpsDgn -O /content/pretrained_models/model_ir_se50.pth

/usr/local/lib/python3.11/dist-packages/gdown/__main__.py:140: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From (original): https://drive.google.com/uc?id=1z_cB187QOc6aqVBdLvYvBjoc93-_EuRm
From (redirected): https://drive.google.com/uc?id=1z_cB187QOc6aqVBdLvYvBjoc93-_EuRm&confirm=t&uuid=3061d757-5756-4f17-8058-681ae6ad092d
To: /content/pretrained_models/restyle_e4e_sg3.pt
100% 809M/809M [00:03<00:00, 244MB/s]
/usr/local/lib/python3.11/dist-packages/gdown/__main__.py:140: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From (original): https://drive.google.com/uc?id=13q6m-bpe3Ws9en9y45JEx2PHQirStt8N
From (redirected): https://drive.google.com/uc?id=13q6m-bpe3Ws9en9y45JEx2PHQirStt8N&confirm=t&uuid=9909ac68-a1c5-4507-9fdb-b25294fc1085
To: 

In [ ]:
!mkdir -p /content/stylegan3-editing/pretrained_models
!cp /content/pretrained_models/shape_predictor_68_face_landmarks.dat /content/stylegan3-editing/pretrained_models/

# preprocess frames

In [ ]:
#@title 단일 프로세스 프레임 추출 코드
import cv2
import os
from glob import glob
from tqdm import tqdm

video_dir = "/content/input_videos"  #@param {type:"string"}
output_base = "/content/extract_frames"  #@param {type:"string"}
extract_per_sec = 7  #@param {type:"integer"}

os.makedirs(output_base, exist_ok=True)

def extract_video_frames(video_path):
    video_name = os.path.splitext(os.path.basename(video_path))[0]
    output_dir = os.path.join(output_base, video_name)
    os.makedirs(output_dir, exist_ok=True)

    cap = cv2.VideoCapture(video_path)
    fps = cap.get(cv2.CAP_PROP_FPS)
    interval = max(1, int(fps // extract_per_sec))

    frame_idx = 0
    frame_list = []

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        if frame_idx % interval == 0:
            frame_list.append(frame)
        frame_idx += 1

    cap.release()

    # 여기서 한 번에 저장!
    for saved_idx, frame in enumerate(frame_list):
        frame_path = os.path.join(output_dir, f"key_{saved_idx:04d}.jpg")
        cv2.imwrite(frame_path, frame)

    return f"{video_name}: {len(frame_list)} frames saved"

# 실행
video_paths = glob(os.path.join(video_dir, "*.mp4"))
print(f"🎬 총 {len(video_paths)}개의 영상 처리 시작")

for video_path in tqdm(video_paths, desc="📦 Processing videos"):
    msg = extract_video_frames(video_path)

🎬 총 3개의 영상 처리 시작


📦 Processing videos: 100%|██████████| 3/3 [00:01<00:00,  1.84it/s]


In [ ]:
#@title 피치 제한 강화 + 눈뜸 50% 기준 + KeyError 해결 코드
import cv2, os, math, numpy as np, pandas as pd
from glob import glob
from tqdm import tqdm
import mediapipe as mp

# MediaPipe 초기화
mp_face_mesh = mp.solutions.face_mesh
FACE_MESH = mp_face_mesh.FaceMesh(
    static_image_mode=True,
    max_num_faces=1,
    refine_landmarks=True,
    min_detection_confidence=0.7,
    min_tracking_confidence=0.5
)

# 눈 좌표 인덱스
LEFT_EYE_IDX = [
    33, 7, 163, 144, 145, 153, 154, 155, 133, 173, 157, 158, 159, 160, 161, 246,
    33, 173, 157, 158, 159, 160, 161, 246, 33
]
RIGHT_EYE_IDX = [
    362, 398, 384, 385, 386, 387, 388, 466, 263, 249, 390, 373, 374, 380, 381, 382,
    362, 398, 384, 385, 386, 387, 388, 466, 362
]

CORE_LEFT_EYE = [33, 160, 158, 133, 153, 144]
CORE_RIGHT_EYE = [362, 385, 387, 263, 373, 380]
LEFT_EYE_VERTICAL = [159, 145]
RIGHT_EYE_VERTICAL = [386, 374]
POSE_IDX = [1, 152, 33, 263, 61, 291]

# 파라미터 설정 - 피치 제한 강화, 눈뜸 기준 완화
YAW_T = 25
PITCH_T = 25  # 35에서 25로 강화 (피치 관대하게 하지 않음)
HEAD_DOWN_BONUS = 1  # 보너스 최소화
ENABLE_ADAPTIVE_EAR = True
EAR_PERCENTILE_HIGH = 37 # 상위 50%를 눈뜬 상태로 판정 (기존 40%에서 완화)
EAR_PERCENTILE_LOW = 10

model_points = np.array([
    (0.0, 0.0, 0.0),
    (0.0, -330.0, -65.0),
    (-225.0, 170.0, -135.0),
    (225.0, 170.0, -135.0),
    (-150.0, -150.0, -125.0),
    (150.0, -150.0, -125.0)
], dtype="double")

def get_mediapipe_landmarks(img):
    h, w = img.shape[:2]
    rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    res = FACE_MESH.process(rgb)
    if not res.multi_face_landmarks:
        return None
    lm = res.multi_face_landmarks[0]
    coords = np.array([[p.x * w, p.y * h] for p in lm.landmark])
    return coords

def estimate_pose_mediapipe(landmarks, img_shape):
    image_points = landmarks[POSE_IDX]
    focal = img_shape[1]
    center = (img_shape[1]/2, img_shape[0]/2)
    cam = np.array([[focal, 0, center[0]], [0, focal, center[1]], [0, 0, 1]], dtype="double")
    dist = np.zeros((4,1))
    success, rv, _ = cv2.solvePnP(model_points, image_points, cam, dist)
    return rv if success else None

def rotation_vector_to_euler(rv):
    rmat, _ = cv2.Rodrigues(rv)
    proj = np.hstack((rmat, np.zeros((3,1))))
    angles = cv2.decomposeProjectionMatrix(proj)[6]
    pitch = math.degrees(math.asin(math.sin(math.radians(angles[1][0]))))
    yaw   = math.degrees(math.asin(math.sin(math.radians(angles[2][0]))))
    roll  = -math.degrees(math.asin(math.sin(math.radians(angles[0][0]))))
    return pitch, yaw, roll

def eye_aspect_ratio(eye):
    A = np.linalg.norm(eye[1] - eye[5])
    B = np.linalg.norm(eye[2] - eye[4])
    C = np.linalg.norm(eye[0] - eye[3])
    return (A + B) / (2.0 * C)

def enhanced_eye_aspect_ratio(landmarks, is_left=True):
    if is_left:
        outer = landmarks[33]
        inner = landmarks[133]
        v1 = np.linalg.norm(landmarks[159] - landmarks[145])
        v2 = np.linalg.norm(landmarks[158] - landmarks[153])
        v3 = np.linalg.norm(landmarks[160] - landmarks[144])
    else:
        outer = landmarks[362]
        inner = landmarks[263]
        v1 = np.linalg.norm(landmarks[386] - landmarks[374])
        v2 = np.linalg.norm(landmarks[385] - landmarks[373])
        v3 = np.linalg.norm(landmarks[387] - landmarks[380])

    horizontal = np.linalg.norm(outer - inner)
    avg_vertical = (v1 + v2 + v3) / 3.0
    return avg_vertical / horizontal

def calculate_adaptive_ear_thresholds(all_ear_values):
    """영상별 적응형 EAR 임계값 계산 - 50% 기준 적용"""
    if len(all_ear_values) < 5:
        return {
            'high_threshold': 0.23,
            'medium_threshold': 0.18,
            'low_threshold': 0.15,
            'min_threshold': 0.12
        }

    ear_array = np.array(all_ear_values)

    # 50% 기준으로 임계값 설정
    high_threshold = np.percentile(ear_array, 100 - EAR_PERCENTILE_HIGH)  # 상위 50%
    medium_threshold = np.percentile(ear_array, 100 - EAR_PERCENTILE_HIGH * 1.2)  # 상위 60%
    low_threshold = np.percentile(ear_array, 100 - EAR_PERCENTILE_HIGH * 1.5)  # 상위 75%
    min_threshold = np.percentile(ear_array, EAR_PERCENTILE_LOW)  # 하위 10%

    # 최소값 보장
    high_threshold = max(high_threshold, 0.16)  # 더 관대하게
    medium_threshold = max(medium_threshold, 0.13)
    low_threshold = max(low_threshold, 0.10)
    min_threshold = max(min_threshold, 0.07)

    return {
        'high_threshold': high_threshold,
        'medium_threshold': medium_threshold,
        'low_threshold': low_threshold,
        'min_threshold': min_threshold,
        'ear_stats': {
            'mean': np.mean(ear_array),
            'std': np.std(ear_array),
            'min': np.min(ear_array),
            'max': np.max(ear_array),
            'count': len(ear_array)
        }
    }

def is_eye_open_adaptive(landmarks, thresholds, pitch_angle=0):
    """적응형 EAR 기반 눈뜸 판정 - 피치 조정 최소화"""

    left_eye_basic = landmarks[CORE_LEFT_EYE]
    right_eye_basic = landmarks[CORE_RIGHT_EYE]

    basic_left_ear = eye_aspect_ratio(left_eye_basic)
    basic_right_ear = eye_aspect_ratio(right_eye_basic)
    basic_avg_ear = (basic_left_ear + basic_right_ear) / 2.0

    enhanced_left_ear = enhanced_eye_aspect_ratio(landmarks, True)
    enhanced_right_ear = enhanced_eye_aspect_ratio(landmarks, False)
    enhanced_avg_ear = (enhanced_left_ear + enhanced_right_ear) / 2.0

    ear_difference = abs(basic_left_ear - basic_right_ear)

    # 피치 조정 최소화 (관대하게 하지 않음)
    pitch_factor = 1.0
    if pitch_angle > 20:  # 20도 이상에서만 최소 조정
        pitch_factor = max(0.95, 1.0 - (pitch_angle - 20) * 0.005)  # 최대 5%만 완화

    # 적응형 임계값 적용
    adj_high = thresholds['high_threshold'] * pitch_factor
    adj_medium = thresholds['medium_threshold'] * pitch_factor
    adj_low = thresholds['low_threshold'] * pitch_factor
    adj_min = thresholds['min_threshold'] * pitch_factor

    # 4단계 눈뜸 판정
    level_1 = (basic_avg_ear > adj_high and
               enhanced_avg_ear > adj_high * 0.9 and
               basic_left_ear > adj_medium and
               basic_right_ear > adj_medium and
               ear_difference < 0.08)

    level_2 = (basic_avg_ear > adj_medium and
               enhanced_avg_ear > adj_medium * 0.8 and
               basic_left_ear > adj_low and
               basic_right_ear > adj_low and
               ear_difference < 0.12)

    level_3 = (basic_avg_ear > adj_low and
               basic_left_ear > adj_min and
               basic_right_ear > adj_min and
               ear_difference < 0.15)

    level_4 = (basic_avg_ear > adj_min and
               basic_left_ear > adj_min * 0.8 and
               basic_right_ear > adj_min * 0.8)

    # 레벨별 점수 부여
    if level_1:
        eye_level = 4
    elif level_2:
        eye_level = 3
    elif level_3:
        eye_level = 2
    elif level_4:
        eye_level = 1
    else:
        eye_level = 0

    return eye_level, {
        'basic_ear': basic_avg_ear,
        'enhanced_ear': enhanced_avg_ear,
        'left_ear': basic_left_ear,
        'right_ear': basic_right_ear,
        'ear_diff': ear_difference,
        'eye_level': eye_level,
        'pitch_factor': pitch_factor,
        'thresholds_used': {
            'high': adj_high,
            'medium': adj_medium,
            'low': adj_low,
            'min': adj_min
        }
    }

def frontal_score_strict_pitch(c):
    """피치 제한 강화된 정면성 평가 함수"""

    yaw_angle = abs(c['yaw'])
    pitch_angle = c['pitch']

    # 엄격한 각도 제한 (피치 관대하게 하지 않음)
    if yaw_angle > YAW_T:  # 25도
        return -1000
    if abs(pitch_angle) > PITCH_T:  # ±25도 (엄격)
        return -1000

    # 기본 페널티 (피치에 더 큰 가중치)
    yaw_penalty = yaw_angle * 0.8
    pitch_penalty = abs(pitch_angle) * 1.2  # 피치 페널티 증가

    # 고개 숙임 보너스 최소화
    head_down_bonus = 0
    if -15 <= pitch_angle <= -5:  # 아주 제한적인 범위에서만
        head_down_bonus = HEAD_DOWN_BONUS * 0.5  # 보너스도 절반으로

    # 눈뜸 레벨 보너스
    eye_level = c.get('eye_level', 0)
    eye_bonus = eye_level * 12  # 눈뜸이 더 중요

    bonus = eye_bonus + (head_down_bonus if pitch_angle < 0 else 0)

    return -(0.5 * yaw_penalty + 0.5 * pitch_penalty) + bonus

def calculate_final_quality_score(pitch, yaw, eye_level, ear_details):
    """최종 품질 점수 계산 - 피치 페널티 강화"""

    # 각도 점수 (피치에 더 큰 페널티)
    yaw_score = max(0, 100 - abs(yaw) * 2.0)
    pitch_score = max(0, 100 - abs(pitch) * 2.5)  # 피치 페널티 증가
    angle_score = (yaw_score + pitch_score) / 2

    # 눈뜸 레벨 점수 (50% 기준이므로 더 관대)
    eye_score = eye_level * 25

    # EAR 품질 점수
    ear_quality = min(100, ear_details['basic_ear'] * 300)

    # 종합 점수 (눈뜸 비중 증가)
    total_score = (
        angle_score * 0.3 +    # 각도 30%
        eye_score * 0.5 +      # 눈뜸 50% (증가)
        ear_quality * 0.2      # EAR 품질 20%
    )

    return total_score

# 경로 설정
INPUT_ROOT = "/content/extract_frames"
OUTPUT_ROOT = "/content/alignmented_frame"
os.makedirs(OUTPUT_ROOT, exist_ok=True)

file_ext = ".jpg"
video_dirs = [d for d in os.listdir(INPUT_ROOT) if os.path.isdir(os.path.join(INPUT_ROOT, d))]
missing_videos = []
detailed_log = []
best_images = []
best_names = []

print(f"📦 총 {len(video_dirs)}개 영상 처리 시작")
print(f"🎯 눈뜸 기준: 상위 {EAR_PERCENTILE_HIGH}% (50% 기준으로 완화)")
print(f"📐 각도 제한: Yaw ±{YAW_T}°, Pitch ±{PITCH_T}° (피치 제한 강화)")
print(f"🎁 고개 숙임 보너스: {HEAD_DOWN_BONUS}점 (최소화)")

for video_name in tqdm(video_dirs, desc="🎯 Strict pitch + 50% eye threshold"):
    input_dir = os.path.join(INPUT_ROOT, video_name)

    # 1단계: 모든 프레임의 EAR 값 수집
    all_ear_values = []
    frame_data = []

    for f in sorted(os.listdir(input_dir)):
        if not f.lower().endswith(file_ext):
            continue

        full_path = os.path.join(input_dir, f)
        img = cv2.imread(full_path)
        if img is None:
            continue

        lm = get_mediapipe_landmarks(img)
        if lm is None:
            continue

        rv = estimate_pose_mediapipe(lm, img.shape)
        if rv is None:
            continue

        pitch, yaw, _ = rotation_vector_to_euler(rv)

        # 엄격한 각도 제한
        if abs(yaw) > YAW_T * 1.5 or abs(pitch) > PITCH_T * 1.5:
            continue

        # EAR 계산
        left_eye = lm[CORE_LEFT_EYE]
        right_eye = lm[CORE_RIGHT_EYE]
        left_ear = eye_aspect_ratio(left_eye)
        right_ear = eye_aspect_ratio(right_eye)
        avg_ear = (left_ear + right_ear) / 2.0

        all_ear_values.append(avg_ear)

        # 프레임 데이터 저장
        x1, y1 = lm[:,0].min(), lm[:,1].min()
        x2, y2 = lm[:,0].max(), lm[:,1].max()
        face_area = (x2 - x1) * (y2 - y1)
        cx, cy = (x1 + x2) / 2, (y1 + y2) / 2

        frame_data.append({
            'img': img,
            'filename': f,
            'landmarks': lm,
            'pitch': pitch,
            'yaw': yaw,
            'avg_ear': avg_ear,
            'cx': cx,
            'cy': cy,
            'face_area': face_area
        })

    if not all_ear_values:
        missing_videos.append(video_name)
        detailed_log.append({
            'video_name': video_name,
            'total_frames': 0,
            'selected': False,
            'selection_level': 0,  # KeyError 방지를 위해 추가
            'reason': 'No valid frames found'
        })
        continue

    # 2단계: 적응형 임계값 계산 (50% 기준)
    thresholds = calculate_adaptive_ear_thresholds(all_ear_values)

    # 3단계: 4단계 후보 분류
    level_4_candidates = []
    level_3_candidates = []
    level_2_candidates = []
    level_1_candidates = []

    for frame in frame_data:
        # 엄격한 피치 제한 적용
        if abs(frame['pitch']) > PITCH_T:
            continue

        eye_level, eye_details = is_eye_open_adaptive(
            frame['landmarks'], thresholds, abs(frame['pitch'])
        )

        if eye_level == 0:
            continue

        candidate = {
            'img': frame['img'],
            'filename': frame['filename'],
            'pitch': frame['pitch'],
            'yaw': frame['yaw'],
            'cx': frame['cx'],
            'cy': frame['cy'],
            'face_area': frame['face_area'],
            'w': frame['img'].shape[1],
            'h': frame['img'].shape[0],
            'eye_level': eye_level,
            'eye_details': eye_details,
            'quality_score': calculate_final_quality_score(
                frame['pitch'], frame['yaw'], eye_level, eye_details
            )
        }

        if eye_level == 4:
            level_4_candidates.append(candidate)
        elif eye_level == 3:
            level_3_candidates.append(candidate)
        elif eye_level == 2:
            level_2_candidates.append(candidate)
        else:
            level_1_candidates.append(candidate)

    # 4단계: 최적 프레임 선택
    best_img = None
    best_filename = None
    selection_reason = "No suitable frames"
    selection_level = 0

    if level_4_candidates:
        best = max(level_4_candidates, key=frontal_score_strict_pitch)
        best_img = best['img']
        best_filename = best['filename']
        selection_reason = f"Level 4: Selected from {len(level_4_candidates)} highest quality candidates"
        selection_level = 4

    elif level_3_candidates:
        best = max(level_3_candidates, key=frontal_score_strict_pitch)
        best_img = best['img']
        best_filename = best['filename']
        selection_reason = f"Level 3: Selected from {len(level_3_candidates)} high quality candidates"
        selection_level = 3

    elif level_2_candidates:
        best = max(level_2_candidates, key=frontal_score_strict_pitch)
        best_img = best['img']
        best_filename = best['filename']
        selection_reason = f"Level 2: Selected from {len(level_2_candidates)} medium quality candidates"
        selection_level = 2

    elif level_1_candidates:
        best = max(level_1_candidates, key=lambda x: x['quality_score'])
        best_img = best['img']
        best_filename = best['filename']
        selection_reason = f"Level 1: Selected from {len(level_1_candidates)} minimum quality candidates"
        selection_level = 1

    elif frame_data:
        # 최후의 수단: 가장 품질 좋은 프레임 무조건 선택
        best_frame = max(frame_data, key=lambda x: x['avg_ear'])
        best_img = best_frame['img']
        best_filename = best_frame['filename']
        selection_reason = f"Emergency: Selected best EAR frame ({best_frame['avg_ear']:.3f})"
        selection_level = 0

    # KeyError 방지를 위한 안전한 로그 기록
    detailed_log.append({
        'video_name': video_name,
        'total_frames': len(frame_data),
        'level_4_candidates': len(level_4_candidates),
        'level_3_candidates': len(level_3_candidates),
        'level_2_candidates': len(level_2_candidates),
        'level_1_candidates': len(level_1_candidates),
        'selected': best_filename is not None,
        'selection_level': selection_level,  # 항상 포함
        'best_filename': best_filename,
        'reason': selection_reason,
        'ear_thresholds': thresholds,
        'quality_score': best.get('quality_score', 0) if 'best' in locals() else 0
    })

    # 프레임 저장
    if best_img is not None:
        save_path = os.path.join(OUTPUT_ROOT, f"{video_name}.jpg")
        cv2.imwrite(save_path, best_img)

        if selection_level <= 1:
            print(f"⚠️  {video_name}: Low quality selection (Level {selection_level})")
    else:
        missing_videos.append(video_name)

# 결과 저장
if missing_videos:
    df_missing = pd.DataFrame(missing_videos, columns=["video_name"])
    df_missing.to_csv("no_frame_found.csv", index=False)
    print(f"❗ {len(missing_videos)}개 영상에서 프레임을 찾지 못함")

df_log = pd.DataFrame(detailed_log)
df_log.to_csv("strict_pitch_50percent_eye_log.csv", index=False)
print(f"📊 처리 결과 저장: strict_pitch_50percent_eye_log.csv")

# KeyError 방지된 안전한 통계 출력
success_rate = (len(video_dirs) - len(missing_videos)) / len(video_dirs) * 100 if video_dirs else 0

# .get() 메서드 사용으로 KeyError 방지
level_4_usage = sum(1 for log in detailed_log if log.get('selection_level', 0) == 4)
level_3_usage = sum(1 for log in detailed_log if log.get('selection_level', 0) == 3)
level_2_usage = sum(1 for log in detailed_log if log.get('selection_level', 0) == 2)
level_1_usage = sum(1 for log in detailed_log if log.get('selection_level', 0) == 1)
emergency_usage = sum(1 for log in detailed_log if log.get('selection_level', 0) == 0)

print(f"✅ 성공률: {success_rate:.1f}% ({len(video_dirs) - len(missing_videos)}/{len(video_dirs)})")
print(f"🏆 Level 4 (최고품질): {level_4_usage}개")
print(f"🥈 Level 3 (고품질): {level_3_usage}개")
print(f"🥉 Level 2 (중품질): {level_2_usage}개")
print(f"📉 Level 1 (최소품질): {level_1_usage}개")
print(f"🚨 Emergency (강제선택): {emergency_usage}개")

print(f"\n📈 품질 분포:")
print(f"   - 고품질 이상 (Level 3+): {(level_4_usage + level_3_usage) / len(video_dirs) * 100:.1f}%")
print(f"   - 중품질 이상 (Level 2+): {(level_4_usage + level_3_usage + level_2_usage) / len(video_dirs) * 100:.1f}%")
print(f"   - 최소품질 이상 (Level 1+): {(len(video_dirs) - emergency_usage) / len(video_dirs) * 100:.1f}%")

print(f"\n🎯 설정 요약:")
print(f"   - 피치 제한: ±{PITCH_T}° (엄격)")
print(f"   - 눈뜸 기준: 상위 {EAR_PERCENTILE_HIGH}% (관대)")
print(f"   - 고개 숙임 보너스: {HEAD_DOWN_BONUS}점 (최소)")


📦 총 3개 영상 처리 시작
🎯 눈뜸 기준: 상위 37% (50% 기준으로 완화)
📐 각도 제한: Yaw ±25°, Pitch ±25° (피치 제한 강화)
🎁 고개 숙임 보너스: 1점 (최소화)


🎯 Strict pitch + 50% eye threshold: 100%|██████████| 3/3 [00:04<00:00,  1.46s/it]

📊 처리 결과 저장: strict_pitch_50percent_eye_log.csv
✅ 성공률: 100.0% (3/3)
🏆 Level 4 (최고품질): 1개
🥈 Level 3 (고품질): 0개
🥉 Level 2 (중품질): 2개
📉 Level 1 (최소품질): 0개
🚨 Emergency (강제선택): 0개

📈 품질 분포:
   - 고품질 이상 (Level 3+): 33.3%
   - 중품질 이상 (Level 2+): 100.0%
   - 최소품질 이상 (Level 1+): 100.0%

🎯 설정 요약:
   - 피치 제한: ±25° (엄격)
   - 눈뜸 기준: 상위 37% (관대)
   - 고개 숙임 보너스: 1점 (최소)


In [ ]:
import numpy
print(numpy.__version__)

1.26.4


# image crop

In [ ]:
%cd /content/stylegan3-editing

/content/stylegan3-editing


In [ ]:
import sys
sys.path.append('/content/stylegan3-editing')

In [ ]:
import time
base_raw_root = "/content/alignmented_frame"
aligned_root = f"{base_raw_root}_aligned"
cropped_root = f"{base_raw_root}_croped"
transform_root = f"{base_raw_root}_transforms"

print("🚀 Aligning all images...")
# 실행 명령어에 PYTHONPATH를 추가하여 모듈을 찾을 경로를 알려줍니다.
!PYTHONPATH=/content/stylegan3-editing python /content/stylegan3-editing/prepare_data/preparing_faces_parallel.py \
    --mode align \
    --root_path "{base_raw_root}"

time.sleep(3)

print("🔁 Cropping all images...")
# 여기도 동일하게 추가합니다.
!PYTHONPATH=/content/stylegan3-editing python /content/stylegan3-editing/prepare_data/preparing_faces_parallel.py \
    --mode crop \
    --root_path "{base_raw_root}" \
    --random_shift 0.05

time.sleep(3)

print("🔁 Computing transforms for all images...")
# 여기도 동일하게 추가합니다.
!PYTHONPATH=/content/stylegan3-editing python /content/stylegan3-editing/prepare_data/compute_landmarks_transforms.py \
    --raw_root "{base_raw_root}" \
    --aligned_root "{aligned_root}" \
    --cropped_root "{cropped_root}" \
    --output_root "{transform_root}"


🚀 Aligning all images...
1
Running on 3 paths
Here we goooo
	ForkPoolWorker-1 is starting to extract on #3 images
	Done!
Mischief managed in -2.4075844287872314s
🔁 Cropping all images...
1
Running on 3 paths
Here we goooo
	ForkPoolWorker-1 is starting to extract on #3 images
	Done!
Mischief managed in -2.117825984954834s
🔁 Computing transforms for all images...
Computing landmarks transforms...
100% 3/3 [00:06<00:00,  2.09s/it]


In [ ]:
import os

# 디렉토리 설정
input_root = "/content/alignmented_frame_croped"
transforms_root = "/content/alignmented_frame_transforms/landmarks_transforms.npy"
output_root = "/content/experiments/restyle_e4e_sg3"
ckpt_path = "/content/pretrained_models/restyle_e4e_sg3.pt"
script_path = "/content/stylegan3-editing/inversion/scripts/inference_iterative.py"

# output 디렉토리 생성
os.makedirs(output_root, exist_ok=True)

print("🚀 Inverting video")

!python {script_path} \
    --output_path "{output_root}" \
    --checkpoint_path "{ckpt_path}" \
    --data_path "{input_root}" \
    --test_batch_size 4 \
    --test_workers 4 \
    --n_iters_per_batch 3 \
    --landmarks_transforms_path "{transforms_root}"

🚀 Inverting video
Loading ReStyle e4e from checkpoint: /content/pretrained_models/restyle_e4e_sg3.pt
Loading StyleGAN3 generator from path: None
Done!
Model successfully loaded!
Loading dataset for ffhq_encode
Setting up PyTorch plugin "filtered_lrelu_plugin"... Done.
100% 1/1 [00:01<00:00,  1.72s/it]
Runtime 1.0465+-0.0000


# save CSV

In [ ]:
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
import pandas as pd

# 1. Load latent dictionary
latent_path = "/content/experiments/restyle_e4e_sg3/latents.npy"
latent_dict = np.load(latent_path, allow_pickle=True).item()

# 2. 정렬된 파일 리스트 확보
filenames = sorted(latent_dict.keys())

# 3. 각 latent에서 마지막 step → 평균 → (512,)
latents = []
for key in filenames:
    latent = latent_dict[key][-1]  # 마지막 step (18, 512)
    mean_latent = latent.mean(axis=0).astype('float32')  # (512,)
    latents.append(mean_latent)

latents = np.stack(latents)  # shape: (N, 512)

# 4. cosine similarity matrix
sim_matrix = cosine_similarity(latents)  # shape: (N, N)

# 5. 각 query 파일마다 top-3 유사한 match + score 저장
rows = []

for i in range(len(filenames)):
    sims = sim_matrix[i].copy()
    sims[i] = -np.inf  # 자기 자신 제외
    top3_idx = np.argsort(sims)[::-1][:3]
    row = {
        "query": filenames[i],
        "top1": filenames[top3_idx[0]],
        "top2": filenames[top3_idx[1]],
        "top3": filenames[top3_idx[2]],
        "top1val": round(float(sims[top3_idx[0]]), 6),
        "top2val": round(float(sims[top3_idx[1]]), 6),
        "top3val": round(float(sims[top3_idx[2]]), 6),
    }
    rows.append(row)

# 6. Save to CSV
df = pd.DataFrame(rows)
df.to_csv("video_similarity_top3_compact.csv", index=False)

print("✅ Saved to video_similarity_top3_compact.csv")


✅ Saved to video_similarity_top3_compact.csv


In [ ]:
!pip install --force-reinstall "numpy<2.0"

  Using cached numpy-1.26.4-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (61 kB)
Using cached numpy-1.26.4-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (18.3 MB)
  Attempting uninstall: numpy
    Found existing installation: numpy 1.26.4
    Uninstalling numpy-1.26.4:
      Successfully uninstalled numpy-1.26.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
ydf 0.12.0 requires protobuf<6.0.0,>=5.29.1, but you have protobuf 3.20.3 which is incompatible.
tensorflow 2.18.0 requires flatbuffers>=24.3.25, but you have flatbuffers 23.5.26 which is incompatible.
thinc 8.3.6 requires numpy<3.0.0,>=2.0.0, but you have numpy 1.26.4 which is incompatible.


In [ ]:
%cd /content
!git clone -q https://github.com/woctezuma/SimSwap.git SimSwap

!pip install -q torch==2.0.1 torchvision==0.15.2 --index-url https://download.pytorch.org/whl/cu118
!pip install -q insightface==0.7.3 onnxruntime moviepy opencv-python imageio==2.34.0
!pip install scikit-video

/content
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 GB 983.7 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.1/6.1 MB 91.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.3/63.3 MB 37.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.3/132.3 kB 12.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchaudio 2.6.0+cu124 requires torch==2.6.0, but you have torch 2.0.1+cu118 which is incompatible.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 439.5/439.5 kB 8.5 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 313.4/313.4 kB 24.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.4/16.4 MB 

In [ ]:
# SimSwap 설정 확인 및 수정

import os

def check_and_fix_simswap():
    """SimSwap 설정 확인 및 수정"""
    print("🔧 SimSwap 설정 확인 중...")

    # 1. 디렉토리 확인
    required_dirs = [
        '/content/SimSwap',
        '/content/arcface_model',
        '/content/SimSwap/checkpoints',
        '/content/SimSwap/parsing_model/checkpoint'
    ]

    for dir_path in required_dirs:
        if os.path.exists(dir_path):
            print(f"✅ {dir_path} 존재")
        else:
            print(f"❌ {dir_path} 없음")

    # 2. 파일 확인
    required_files = [
        '/content/arcface_model/arcface_checkpoint.tar',
        '/content/SimSwap/parsing_model/checkpoint/79999_iter.pth'
    ]

    for file_path in required_files:
        if os.path.exists(file_path):
            print(f"✅ {file_path} 존재")
        else:
            print(f"❌ {file_path} 없음")

    # 3. checkpoints 디렉토리 내용 확인
    checkpoints_dir = '/content/SimSwap/checkpoints'
    if os.path.exists(checkpoints_dir):
        files = os.listdir(checkpoints_dir)
        print(f"📁 checkpoints 디렉토리 내용: {files}")

        # people 폴더 확인
        people_dir = os.path.join(checkpoints_dir, 'people')
        if os.path.exists(people_dir):
            people_files = os.listdir(people_dir)
            print(f"📁 people 폴더 내용: {people_files}")
        else:
            print("❌ people 폴더가 없음")

    # 4. arcface 압축 해제 확인
    arcface_tar = '/content/arcface_model/arcface_checkpoint.tar'
    if os.path.exists(arcface_tar):
        print("🔧 arcface 압축 해제 중...")
        os.chdir('/content/arcface_model')
        os.system('tar -xf arcface_checkpoint.tar')

        # 압축 해제 후 파일 확인
        extracted_files = os.listdir('/content/arcface_model')
        print(f"📁 압축 해제 후 arcface_model 내용: {extracted_files}")

def download_missing_models():
    """누락된 모델 다운로드"""
    print("📥 누락된 모델 다운로드 중...")

    # 기본 디렉토리 생성
    os.makedirs('/content/arcface_model', exist_ok=True)
    os.makedirs('/content/SimSwap/parsing_model/checkpoint', exist_ok=True)
    os.makedirs('/content/SimSwap/checkpoints', exist_ok=True)
    os.makedirs('/content/insightface_func/models', exist_ok=True)

    # 모델 다운로드
    os.system('wget -q https://github.com/woctezuma/SimSwap-colab/releases/download/1.0/arcface_checkpoint.tar -O /content/arcface_model/arcface_checkpoint.tar')
    os.system('wget -q https://github.com/neuralchen/SimSwap/releases/download/1.0/checkpoints.zip -O /content/checkpoints.zip')
    os.system('wget -q https://github.com/neuralchen/SimSwap/releases/download/1.0/79999_iter.pth -O /content/SimSwap/parsing_model/checkpoint/79999_iter.pth')
    os.system('wget -q https://github.com/woctezuma/SimSwap-colab/releases/download/antelope/antelope.zip -O /content/antelope.zip')

    # 압축 해제
    os.system('unzip -q /content/checkpoints.zip -d /content/SimSwap/checkpoints')
    os.system('unzip -q /content/antelope.zip -d /content/insightface_func/models/')

    print("✅ 모델 다운로드 완료")

def fix_simswap_compatibility():
    """SimSwap 호환성 수정"""
    print("🔧 SimSwap 호환성 수정 중...")

    os.chdir('/content/SimSwap')

    # PyTorch 호환성 수정
    os.system("sed -i.bak 's/torch.load(netArc_checkpoint, map_location=torch.device(\"cpu\"))/torch.load(netArc_checkpoint, map_location=torch.device(\"cpu\"), weights_only=False)/' models/fs_model.py")

    # 감지 크기 수정
    os.system("sed -i 's/det_size=(640,640)/det_size=(224,224)/' test_wholeimage_swapsingle.py")

    # NMS 임계값 수정
    with open('test_wholeimage_swapsingle.py', 'r') as f:
        content = f.read()
    if 'app.models["detection"].nms_thresh' not in content:
        new_lines = []
        for line in content.splitlines():
            new_lines.append(line)
            if 'app.prepare(' in line:
                new_lines.append('    if hasattr(app.models, "detection"):')
                new_lines.append('        app.models["detection"].nms_thresh = 0.3')
                new_lines.append('        app.models["detection"].det_thresh = 0.3')
        with open('test_wholeimage_swapsingle.py', 'w') as f:
            f.write('\n'.join(new_lines))

    print("✅ 호환성 수정 완료")

# 실행
print("=== SimSwap 설정 진단 및 수정 ===")
check_and_fix_simswap()
download_missing_models()
check_and_fix_simswap()  # 다시 확인
fix_simswap_compatibility()
print("🎉 SimSwap 설정 완료!")

=== SimSwap 설정 진단 및 수정 ===
🔧 SimSwap 설정 확인 중...
✅ /content/SimSwap 존재
❌ /content/arcface_model 없음
❌ /content/SimSwap/checkpoints 없음
❌ /content/SimSwap/parsing_model/checkpoint 없음
❌ /content/arcface_model/arcface_checkpoint.tar 없음
❌ /content/SimSwap/parsing_model/checkpoint/79999_iter.pth 없음
📥 누락된 모델 다운로드 중...
✅ 모델 다운로드 완료
🔧 SimSwap 설정 확인 중...
✅ /content/SimSwap 존재
✅ /content/arcface_model 존재
✅ /content/SimSwap/checkpoints 존재
✅ /content/SimSwap/parsing_model/checkpoint 존재
✅ /content/arcface_model/arcface_checkpoint.tar 존재
✅ /content/SimSwap/parsing_model/checkpoint/79999_iter.pth 존재
📁 checkpoints 디렉토리 내용: ['people']
📁 people 폴더 내용: ['latest_net_G.pth', 'latest_net_D2.pth', 'latest_net_D1.pth', 'loss_log.txt', 'iter.txt', 'web', 'opt.txt']
🔧 arcface 압축 해제 중...
📁 압축 해제 후 arcface_model 내용: ['arcface_checkpoint.tar']
🔧 SimSwap 호환성 수정 중...
✅ 호환성 수정 완료
🎉 SimSwap 설정 완료!


In [ ]:
# InsightFace 모델 문제 해결

import os

def fix_insightface_models():
    """InsightFace 모델 문제 해결"""
    print("🔧 InsightFace 모델 문제 해결 중...")

    # 1. 현재 상황 확인
    models_dir = '/content/SimSwap/insightface_func/models'
    if os.path.exists(models_dir):
        print(f"📁 models 디렉토리 내용: {os.listdir(models_dir)}")

        antelope_dir = os.path.join(models_dir, 'antelope')
        if os.path.exists(antelope_dir):
            print(f"📁 antelope 디렉토리 내용: {os.listdir(antelope_dir)}")
        else:
            print("❌ antelope 디렉토리 없음")

    # 2. antelope 모델 재다운로드 및 올바른 위치에 설치
    print("📥 antelope 모델 재설치 중...")

    antelope_dir = '/content/SimSwap/insightface_func/models/antelope'

    # 기존 파일 정리
    os.system('rm -rf /content/SimSwap/insightface_func/models/antelope')
    os.system('rm -f /content/antelope.zip')

    # 올바른 antelope 모델 다운로드
    os.system('wget -q https://github.com/woctezuma/SimSwap-colab/releases/download/antelope/antelope.zip -O /content/antelope.zip')

    # 압축 해제
    os.system('unzip -q /content/antelope.zip -d /content/SimSwap/insightface_func/models/')

    # 3. 다시 확인
    if os.path.exists(antelope_dir):
        files = os.listdir(antelope_dir)
        print(f"📁 설치 후 antelope 내용: {files}")

        # 필수 파일들 확인
        required_files = ['glintr100.onnx', 'scrfd_10g_bnkps.onnx']
        for req_file in required_files:
            if req_file in files:
                print(f"✅ {req_file} 존재")
            else:
                print(f"❌ {req_file} 없음")

    print("✅ InsightFace 모델 재설치 완료")

def alternative_download():
    """대안 다운로드 방법"""
    print("🔄 대안 방법으로 모델 다운로드 중...")

    # 개별 파일 다운로드
    antelope_dir = '/content/SimSwap/insightface_func/models/antelope'
    os.makedirs(antelope_dir, exist_ok=True)

    # 필수 모델 파일들 개별 다운로드
    models = {
        'glintr100.onnx': 'https://github.com/deepinsight/insightface/releases/download/v0.7/buffalo_l.zip',
        'scrfd_10g_bnkps.onnx': 'https://github.com/deepinsight/insightface/releases/download/v0.7/buffalo_l.zip'
    }

    # buffalo_l 모델 다운로드 (더 안정적)
    print("📥 buffalo_l 모델 다운로드 중...")
    os.system('wget -q https://github.com/deepinsight/insightface/releases/download/v0.7/buffalo_l.zip -O /content/buffalo_l.zip')
    os.system(f'unzip -q /content/buffalo_l.zip -d {antelope_dir}')

    # 파일 확인
    if os.path.exists(antelope_dir):
        files = os.listdir(antelope_dir)
        print(f"📁 buffalo_l 설치 후: {files}")

# 실행
fix_insightface_models()

# 실패시 대안 방법
if not os.path.exists('/content/SimSwap/insightface_func/models/antelope/glintr100.onnx'):
    print("\n🔄 기본 방법 실패, 대안 방법 시도...")
    alternative_download()

🔧 InsightFace 모델 문제 해결 중...
📥 antelope 모델 재설치 중...
📁 설치 후 antelope 내용: ['glintr100.onnx', 'scrfd_10g_bnkps.onnx']
✅ glintr100.onnx 존재
✅ scrfd_10g_bnkps.onnx 존재
✅ InsightFace 모델 재설치 완료


# 생성 및 보간

## 1단계 프레임 추출

In [ ]:
import cv2
import os
from glob import glob
from tqdm import tqdm
import re

def extract_number_from_filename(filename):
    """파일명에서 숫자 추출 (영상과 이미지 매칭용)"""
    numbers = re.findall(r'\d+', os.path.splitext(filename)[0])
    if numbers:
        return int(numbers[0])
    return None

def extract_frames_all_videos(
    video_dir="/content/input_videos",
    output_base_dir="/content/simswap_results",
    interval=5
):
    # 모든 mp4 파일 검색
    video_files = glob(os.path.join(video_dir, "*.mp4"))
    print(f"총 {len(video_files)}개 mp4 영상에서 프레임 추출 시작!")

    for video_path in video_files:
        video_name = os.path.splitext(os.path.basename(video_path))[0]
        number = extract_number_from_filename(video_name)
        if number is None:
            print(f"  {video_path}: 숫자 미포함, 스킵")
            continue

        # 디렉토리 구조: /content/simswap_results/pair_XXX_xxxx/extracted_frames/
        pair_name = f"pair_{number:03d}_{video_name}"
        pair_output_dir = os.path.join(output_base_dir, pair_name, "extracted_frames")
        os.makedirs(pair_output_dir, exist_ok=True)

        cap = cv2.VideoCapture(video_path)
        if not cap.isOpened():
            print(f"  {video_path}: 오픈 실패, 스킵")
            continue

        total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
        print(f"  {video_name}({number}): {total_frames}프레임 중 interval={interval}마다 추출")

        frame_count = 0
        saved_count = 0
        with tqdm(total=total_frames, desc=f"    {pair_name} 프레임 추출") as pbar:
            while True:
                ret, frame = cap.read()
                if not ret:
                    break

                if frame_count % interval == 0:
                    cv2.imwrite(os.path.join(pair_output_dir, f'frame_{saved_count:04d}.jpg'), frame)
                    saved_count += 1

                frame_count += 1
                pbar.update(1)

        cap.release()
        print(f"    >> 저장: {saved_count}장 완료 ({pair_output_dir})")
    print("모든 mp4 프레임 추출 완료.")

# 실제 사용 예시
if __name__ == "__main__":
    VIDEO_DIR = "/content/input_videos"
    OUTPUT_DIR = "/content/simswap_results"
    INTERVAL = 5
    extract_frames_all_videos(VIDEO_DIR, OUTPUT_DIR, INTERVAL)


총 3개 mp4 영상에서 프레임 추출 시작!
  0555(555): 460프레임 중 interval=5마다 추출


    pair_555_0555 프레임 추출: 100%|██████████| 460/460 [00:00<00:00, 1252.61it/s]


    >> 저장: 92장 완료 (/content/simswap_results/pair_555_0555/extracted_frames)
  0556(556): 582프레임 중 interval=5마다 추출


    pair_556_0556 프레임 추출: 100%|██████████| 582/582 [00:00<00:00, 1315.80it/s]


    >> 저장: 117장 완료 (/content/simswap_results/pair_556_0556/extracted_frames)
  0554(554): 457프레임 중 interval=5마다 추출


    pair_554_0554 프레임 추출: 100%|██████████| 457/457 [00:00<00:00, 1164.71it/s]

    >> 저장: 92장 완료 (/content/simswap_results/pair_554_0554/extracted_frames)
모든 mp4 프레임 추출 완료.


## 2단계 SimSwap 얼굴 교체

In [ ]:
import subprocess
import os
from glob import glob
from tqdm import tqdm
import re
from concurrent.futures import ThreadPoolExecutor, as_completed
import threading
import time


def extract_number_from_filename(filename):
    """파일명에서 숫자 추출 (영상과 이미지 매칭용)"""
    numbers = re.findall(r'\d+', os.path.splitext(filename)[0])
    if numbers:
        return int(numbers[0])
    return None


def get_video_image_pairs(frame_base_dir, image_dir):
    """추출된 프레임과 이미지 파일들을 번호로 매칭"""
    # 프레임 디렉토리들 검색 (pair_XXX_xxxx 형태)
    pair_dirs = glob(os.path.join(frame_base_dir, "pair_*"))

    # 이미지 파일들 검색
    image_files = glob(os.path.join(image_dir, "*.jpg")) + glob(os.path.join(image_dir, "*.png"))

    # 이미지를 번호별로 딕셔너리 생성
    image_dict = {}
    for image_path in image_files:
        filename = os.path.basename(image_path)
        number = extract_number_from_filename(filename)
        if number is not None:
            image_dict[number] = image_path

    # 매칭되는 쌍 찾기
    pairs = []
    for pair_dir in pair_dirs:
        # pair_XXX_xxxx에서 번호 추출
        pair_name = os.path.basename(pair_dir)
        number = extract_number_from_filename(pair_name)

        if number is not None and number in image_dict:
            frame_dir = os.path.join(pair_dir, "extracted_frames")
            if os.path.exists(frame_dir):
                pairs.append({
                    'number': number,
                    'pair_name': pair_name,
                    'frame_dir': frame_dir,
                    'image_path': image_dict[number],
                    'swap_dir': os.path.join(pair_dir, "swapped_frames")
                })

    return pairs


def run_simswap_single_frame(source_img, target_img, result_subdir):
    """단일 프레임에 대한 SimSwap 처리"""
    # SimSwap 경로 설정
    arcface_path = '/content/arcface_model/arcface_checkpoint.tar'
    checkpoints_dir = '/content/SimSwap/checkpoints'

    # 결과 디렉토리 생성
    os.makedirs(result_subdir, exist_ok=True)

    # SimSwap 실행 명령
    cmd = [
        'python', 'test_wholeimage_swapsingle.py',
        '--crop_size', '224',
        '--use_mask',
        '--no_simswaplogo',
        '--name', 'people',
        '--Arc_path', arcface_path,
        '--pic_a_path', source_img,
        '--pic_b_path', target_img,
        '--pic_specific_path', target_img,
        '--output_path', result_subdir,
        '--checkpoints_dir', checkpoints_dir
    ]

    # 원래 디렉토리 저장
    original_cwd = os.getcwd()

    try:
        # SimSwap 디렉토리로 이동
        os.chdir('/content/SimSwap')

        # SimSwap 실행
        result = subprocess.run(cmd, stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL)

        # 결과 파일 확인
        result_files = [f for f in os.listdir(result_subdir) if f.endswith(('.jpg', '.png'))]
        return len(result_files) > 0

    except Exception as e:
        return False
    finally:
        # 원래 디렉토리로 복귀
        os.chdir(original_cwd)


def process_single_pair_no_progress(pair_info, max_workers=4):
    """단일 쌍의 모든 프레임을 진행바 없이 병렬 처리"""
    source_img = pair_info['image_path']
    frame_dir = pair_info['frame_dir']
    swap_dir = pair_info['swap_dir']

    # 프레임 파일들 가져오기
    frames = sorted([f for f in os.listdir(frame_dir) if f.endswith('.jpg')])
    total_frames = len(frames)

    if total_frames == 0:
        return False, 0, total_frames

    # 작업 목록 생성
    tasks = []
    for i, frame_name in enumerate(frames):
        target_img = os.path.join(frame_dir, frame_name)
        result_subdir = os.path.join(swap_dir, f'frame_{i:04d}')
        tasks.append((source_img, target_img, result_subdir, i))

    # 병렬 처리 (진행바 완전 제거)
    success_count = 0

    with ThreadPoolExecutor(max_workers=max_workers) as executor:
        # 작업 제출
        futures = [executor.submit(run_simswap_single_frame, task[0], task[1], task[2]) for task in tasks]

        # 결과 수집 (진행바 없이)
        for future in as_completed(futures):
            try:
                success = future.result()
                if success:
                    success_count += 1
            except Exception:
                pass  # 에러도 조용히 무시

    return success_count > 0, success_count, total_frames


def simswap_all_pairs_parallel(
    frame_base_dir="/content/simswap_results",
    image_dir="/content/experiments/restyle_e4e_sg3/inference_results/0",
    max_workers_per_pair=4,
    max_concurrent_pairs=2
):
    """모든 쌍에 대해 병렬 SimSwap 처리 - 전체 진행상황만 표시"""

    # tqdm 전역 비활성화 (혹시 모를 다른 진행바들까지 차단)
    import tqdm as tqdm_module
    original_tqdm = tqdm_module.tqdm

    def disabled_tqdm(*args, **kwargs):
        kwargs['disable'] = True
        return original_tqdm(*args, **kwargs)

    tqdm_module.tqdm = disabled_tqdm

    try:
        # SimSwap 환경 확인
        if not os.path.exists('/content/SimSwap'):
            print("❌ SimSwap이 설치되지 않았습니다. 먼저 SimSwap을 설치해주세요.")
            return

        # 매칭되는 쌍 찾기
        pairs = get_video_image_pairs(frame_base_dir, image_dir)

        if not pairs:
            print("❌ 매칭되는 프레임-이미지 쌍을 찾을 수 없습니다!")
            return

        # 초기 정보만 출력
        print(f"🎯 총 {len(pairs)}개 쌍 처리 시작 (쌍당 {max_workers_per_pair}워커, 동시 {max_concurrent_pairs}쌍)")

        # 전체 진행상황을 위한 변수들
        completed_pairs = 0
        total_pairs = len(pairs)
        lock = threading.Lock()

        # 전체 진행바 생성 (원래 tqdm 사용)
        overall_pbar = original_tqdm(total=total_pairs, desc="🔄 SimSwap 처리", unit="쌍")

        def process_pair_with_progress(pair):
            """쌍 처리 후 전체 진행바 업데이트"""
            nonlocal completed_pairs

            # 진행바 없는 함수 사용
            success, success_frames, total_frames = process_single_pair_no_progress(pair, max_workers_per_pair)

            # 스레드 안전하게 진행바 업데이트
            with lock:
                completed_pairs += 1
                # 진행바 정보 업데이트
                overall_pbar.set_postfix({
                    '완료': f"{completed_pairs}/{total_pairs}",
                    '현재': pair['pair_name'][:12],  # 이름 길이 제한
                    '성공': f"{success_frames}/{total_frames}" if total_frames > 0 else "0/0"
                })
                overall_pbar.update(1)

            return success, pair

        start_time = time.time()
        success_count = 0

        # 쌍별로 병렬 처리
        with ThreadPoolExecutor(max_workers=max_concurrent_pairs) as executor:
            # 모든 쌍에 대한 작업 제출
            futures = [executor.submit(process_pair_with_progress, pair) for pair in pairs]

            # 결과 수집 (조용히)
            for future in as_completed(futures):
                try:
                    success, pair = future.result()
                    if success:
                        success_count += 1
                except Exception:
                    # 에러가 발생해도 진행바는 계속 업데이트
                    with lock:
                        completed_pairs += 1
                        overall_pbar.set_postfix({
                            '완료': f"{completed_pairs}/{total_pairs}",
                            '상태': '에러'
                        })
                        overall_pbar.update(1)

        # 진행바 닫기
        overall_pbar.close()

        end_time = time.time()
        total_duration = end_time - start_time

        # 최종 결과만 출력
        print(f"🎊 완료! 성공: {success_count}/{len(pairs)}개, 시간: {total_duration:.1f}초")

    finally:
        # tqdm 원래대로 복구
        tqdm_module.tqdm = original_tqdm


# 개별 쌍만 처리하고 싶을 때 사용하는 함수
def simswap_single_pair_only(pair_name, max_workers=4,
                            frame_base_dir="/content/simswap_results",
                            image_dir="/content/experiments/restyle_e4e_sg3/inference_results/0"):
    """특정 쌍만 병렬로 처리"""
    pairs = get_video_image_pairs(frame_base_dir, image_dir)

    target_pair = None
    for pair in pairs:
        if pair['pair_name'] == pair_name:
            target_pair = pair
            break

    if target_pair is None:
        print(f"❌ '{pair_name}' 쌍을 찾을 수 없습니다!")
        return False

    success, success_frames, total_frames = process_single_pair_no_progress(target_pair, max_workers)

    if success:
        print(f"✅ [{pair_name}] 완료: {success_frames}/{total_frames} 프레임 성공")
    else:
        print(f"❌ [{pair_name}] 실패")

    return success


# 실제 사용 예시
if __name__ == "__main__":
    FRAME_BASE_DIR = "/content/simswap_results"
    IMAGE_DIR = "/content/experiments/restyle_e4e_sg3/inference_results/0"

    simswap_all_pairs_parallel(
        frame_base_dir=FRAME_BASE_DIR,
        image_dir=IMAGE_DIR,
        max_workers_per_pair=4,
        max_concurrent_pairs=2
    )


🎯 총 3개 쌍 처리 시작 (쌍당 4워커, 동시 2쌍)


🔄 SimSwap 처리: 100%|██████████| 3/3 [07:58<00:00, 159.60s/쌍, 완료=3/3, 현재=pair_556_055, 성공=117/117]

🎊 완료! 성공: 3/3개, 시간: 478.8초


## 3단계 영상생성

In [ ]:
import cv2
import os
from glob import glob
from tqdm import tqdm
import re
from concurrent.futures import ThreadPoolExecutor, as_completed
import threading


def extract_number_from_filename(filename):
    """파일명에서 숫자 추출 (영상과 이미지 매칭용)"""
    numbers = re.findall(r'\d+', os.path.splitext(filename)[0])
    if numbers:
        return int(numbers[0])
    return None


def get_swapped_pairs_info(frame_base_dir="/content/simswap_results"):
    """SimSwap 완료된 쌍들의 정보 가져오기"""
    pair_dirs = glob(os.path.join(frame_base_dir, "pair_*"))
    pairs_info = []

    for pair_dir in pair_dirs:
        pair_name = os.path.basename(pair_dir)
        number = extract_number_from_filename(pair_name)

        if number is not None:
            swap_dir = os.path.join(pair_dir, "swapped_frames")
            if os.path.exists(swap_dir):
                # 스왑된 프레임 개수 확인
                swap_subdirs = [d for d in os.listdir(swap_dir) if os.path.isdir(os.path.join(swap_dir, d))]
                pairs_info.append({
                    'number': number,
                    'pair_name': pair_name,
                    'pair_dir': pair_dir,
                    'swap_dir': swap_dir,
                    'frame_count': len(swap_subdirs)
                })

    return sorted(pairs_info, key=lambda x: x['number'])


def create_video_from_swapped_frames(swap_dir, output_video, target_fps):
    """SimSwap 결과 프레임들을 비디오로 변환"""
    if not os.path.exists(swap_dir):
        return False, "swap_dir이 존재하지 않음"

    # 결과 이미지들 수집
    result_images = []
    swap_subdirs = sorted([d for d in os.listdir(swap_dir) if os.path.isdir(os.path.join(swap_dir, d))])

    if not swap_subdirs:
        return False, "스왑된 프레임 디렉토리가 없음"

    for subdir in swap_subdirs:
        subdir_path = os.path.join(swap_dir, subdir)
        result_files = [f for f in os.listdir(subdir_path) if f.endswith(('.jpg', '.png'))]
        if result_files:
            # 첫 번째 결과 파일 사용
            result_images.append(os.path.join(subdir_path, result_files[0]))

    if not result_images:
        return False, "결과 이미지가 없음"

    # 첫 번째 이미지로 비디오 크기 결정
    first_img = cv2.imread(result_images[0])
    if first_img is None:
        return False, f"첫 번째 이미지를 읽을 수 없음: {result_images[0]}"

    height, width, _ = first_img.shape

    # 비디오 라이터 설정
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    out = cv2.VideoWriter(output_video, fourcc, target_fps, (width, height))

    if not out.isOpened():
        return False, f"비디오 라이터를 열 수 없음: {output_video}"

    # 프레임들을 비디오로 작성
    success_frames = 0
    for img_path in result_images:
        img = cv2.imread(img_path)
        if img is not None:
            out.write(img)
            success_frames += 1

    out.release()

    if success_frames > 0:
        return True, f"비디오 생성 완료: {success_frames}프레임"
    else:
        return False, "프레임을 읽을 수 없음"


def process_single_pair_video_creation(pair_info, original_fps=30.0, frame_interval=5):
    """단일 쌍의 기본 비디오 생성"""
    swap_dir = pair_info['swap_dir']
    pair_name = pair_info['pair_name']
    pair_dir = pair_info['pair_dir']

    # 프레임 간격에 따른 FPS 계산
    base_fps = original_fps / frame_interval

    # 기본 비디오 생성
    base_video_path = os.path.join(pair_dir, f"{pair_name}_base.mp4")
    success, message = create_video_from_swapped_frames(swap_dir, base_video_path, base_fps)

    if success:
        return True, f"기본 비디오: {base_video_path}"
    else:
        return False, f"비디오 생성 실패: {message}"


def create_videos_all_pairs_parallel(
    frame_base_dir="/content/simswap_results",
    original_fps=30.0,
    frame_interval=5,
    max_workers=3
):
    """모든 쌍에 대해 병렬로 기본 비디오 생성"""

    # 완료된 쌍들 찾기
    pairs = get_swapped_pairs_info(frame_base_dir)

    if not pairs:
        print("❌ SimSwap이 완료된 쌍을 찾을 수 없습니다!")
        print(f"디렉토리 확인: {frame_base_dir}")
        return

    print(f"🎬 총 {len(pairs)}개 쌍의 기본 비디오 생성 시작!")
    print(f"⚙️ 설정: 원본 FPS={original_fps}, 프레임 간격={frame_interval}, 워커={max_workers}개")

    # 전체 진행상황을 위한 변수들
    completed_pairs = 0
    total_pairs = len(pairs)
    lock = threading.Lock()

    # 전체 진행바 생성
    overall_pbar = tqdm(total=total_pairs, desc="🎥 비디오 생성", unit="쌍")

    def process_pair_with_progress(pair):
        """쌍 처리 후 전체 진행바 업데이트"""
        nonlocal completed_pairs

        success, message = process_single_pair_video_creation(pair, original_fps, frame_interval)

        # 스레드 안전하게 진행바 업데이트
        with lock:
            completed_pairs += 1
            overall_pbar.set_postfix({
                '완료': f"{completed_pairs}/{total_pairs}",
                '현재': pair['pair_name'][:12],
                '프레임': f"{pair['frame_count']}개"
            })
            overall_pbar.update(1)

        return success, pair, message

    success_count = 0

    # 병렬 처리
    with ThreadPoolExecutor(max_workers=max_workers) as executor:
        # 모든 쌍에 대한 작업 제출
        futures = [executor.submit(process_pair_with_progress, pair) for pair in pairs]

        # 결과 수집
        for future in as_completed(futures):
            try:
                success, pair, message = future.result()
                if success:
                    success_count += 1
            except Exception as e:
                # 에러가 발생해도 진행바는 계속 업데이트
                with lock:
                    completed_pairs += 1
                    overall_pbar.set_postfix({
                        '완료': f"{completed_pairs}/{total_pairs}",
                        '상태': '에러'
                    })
                    overall_pbar.update(1)

    # 진행바 닫기
    overall_pbar.close()

    print(f"\n🎊 기본 비디오 생성 완료!")
    print(f"📊 성공: {success_count}/{len(pairs)}개")
    print(f"📁 결과 위치: {frame_base_dir}")
    print(f"💡 다음 단계: 프레임 보간을 원한다면 별도 스크립트를 실행하세요")


# 개별 쌍만 처리하고 싶을 때 사용하는 함수
def create_single_pair_video_only(pair_name, original_fps=30.0, frame_interval=5,
                                 frame_base_dir="/content/simswap_results"):
    """특정 쌍만 기본 비디오 생성"""
    pairs = get_swapped_pairs_info(frame_base_dir)

    target_pair = None
    for pair in pairs:
        if pair['pair_name'] == pair_name:
            target_pair = pair
            break

    if target_pair is None:
        print(f"❌ '{pair_name}' 쌍을 찾을 수 없습니다!")
        return False

    print(f"🎬 [{pair_name}] 기본 비디오 생성 시작...")
    success, message = process_single_pair_video_creation(target_pair, original_fps, frame_interval)

    if success:
        print(f"✅ [{pair_name}] {message}")
    else:
        print(f"❌ [{pair_name}] {message}")

    return success


# 실제 사용 예시
if __name__ == "__main__":
    FRAME_BASE_DIR = "/content/simswap_results"  # SimSwap 결과가 있는 디렉토리
    ORIGINAL_FPS = 30.0  # 원본 비디오의 FPS
    FRAME_INTERVAL = 5   # 프레임 추출 시 사용한 간격
    MAX_WORKERS = 3      # 병렬 처리 워커 수

    print("=== SimSwap 3단계: 기본 비디오 생성 시작 ===")
    print(f"결과 디렉토리: {FRAME_BASE_DIR}")
    print(f"원본 FPS: {ORIGINAL_FPS}")
    print(f"프레임 간격: {FRAME_INTERVAL}")

    # 모든 쌍의 기본 비디오 생성
    create_videos_all_pairs_parallel(
        frame_base_dir=FRAME_BASE_DIR,
        original_fps=ORIGINAL_FPS,
        frame_interval=FRAME_INTERVAL,
        max_workers=MAX_WORKERS
    )

    # 특정 쌍만 처리하고 싶다면:
    # create_single_pair_video_only("pair_001_video1", original_fps=30.0, frame_interval=5)


=== SimSwap 3단계: 기본 비디오 생성 시작 ===
결과 디렉토리: /content/simswap_results
원본 FPS: 30.0
프레임 간격: 5
🎬 총 3개 쌍의 기본 비디오 생성 시작!
⚙️ 설정: 원본 FPS=30.0, 프레임 간격=5, 워커=3개


🎥 비디오 생성: 100%|██████████| 3/3 [00:00<00:00,  5.61쌍/s, 완료=3/3, 현재=pair_556_055, 프레임=117개]


🎊 기본 비디오 생성 완료!
📊 성공: 3/3개
📁 결과 위치: /content/simswap_results
💡 다음 단계: 프레임 보간을 원한다면 별도 스크립트를 실행하세요


In [ ]:

%cd /content

!git clone https://github.com/hzwer/Practical-RIFE.git
%cd Practical-RIFE

!pip install -r requirements.txt
!pip install -q gdown opencv-python numpy tqdm

!gdown --fuzzy "https://drive.google.com/file/d/1l5u6G8vEkPAT7cYYWwzB6OG8vwBYrxiS/view" -O RIFE_trained_model_HDv3.zip
!unzip -q RIFE_trained_model_HDv3.zip
!mv RIFEv4.21/train_log ./


/content
fatal: destination path 'Practical-RIFE' already exists and is not an empty directory.
/content/Practical-RIFE
Downloading...
From (original): https://drive.google.com/uc?id=1l5u6G8vEkPAT7cYYWwzB6OG8vwBYrxiS
From (redirected): https://drive.google.com/uc?id=1l5u6G8vEkPAT7cYYWwzB6OG8vwBYrxiS&confirm=t&uuid=7e83813f-d126-4471-8a50-cf4ccc028ca6
To: /content/Practical-RIFE/RIFE_trained_model_HDv3.zip
100% 38.1M/38.1M [00:00<00:00, 142MB/s]
replace __MACOSX/._RIFEv4.21? [y]es, [n]o, [A]ll, [N]one, [r]ename: N
mv: cannot move 'RIFEv4.21/train_log' to './train_log': Directory not empty


## 4단계 보간


In [ ]:
import os
from glob import glob
from tqdm import tqdm
import subprocess
import cv2

RIFE_DIR = "/content/Practical-RIFE"
SIMSWAP_RESULT_DIR = "/content/simswap_results"
TARGET_FPS = 30
CUDA_DEVICE = "0"

def interpolate_video_rife(input_path, output_path, fps=30):
    cmd = [
        "python", "inference_video.py",
        "--video", input_path,
        "--output", output_path,
        "--fps", str(fps),
    ]
    env = os.environ.copy()
    env["CUDA_VISIBLE_DEVICES"] = CUDA_DEVICE
    result = subprocess.run(cmd, cwd=RIFE_DIR, capture_output=True, text=True)
    return result.returncode == 0, result.stdout.strip(), result.stderr.strip()

def match_video_duration(original_path, input_path, output_path):
    """원본 영상 길이에 맞춰 RIFE 보간 영상의 속도 조정"""
    original_cap = cv2.VideoCapture(original_path)
    input_cap = cv2.VideoCapture(input_path)

    original_duration = original_cap.get(cv2.CAP_PROP_FRAME_COUNT) / original_cap.get(cv2.CAP_PROP_FPS)
    input_duration = input_cap.get(cv2.CAP_PROP_FRAME_COUNT) / input_cap.get(cv2.CAP_PROP_FPS)

    original_cap.release()
    input_cap.release()

    if input_duration == 0:
        print(f"⚠️ {input_path} 길이 계산 실패")
        return False

    # ❗ 중요: 원래 길이에 맞춰 느리게 재생
    speed_ratio = original_duration / input_duration

    # ffmpeg로 재생 시간 맞추기
    cmd = [
        "ffmpeg", "-y",
        "-i", input_path,
        "-filter:v", f"setpts={speed_ratio}*PTS",
        "-an",
        output_path
    ]
    subprocess.run(cmd, capture_output=True)
    return os.path.exists(output_path)

# base.mp4 파일 목록
base_videos = sorted(glob(os.path.join(SIMSWAP_RESULT_DIR, "pair_*", "*_base.mp4")))

if not base_videos:
    print("❌ base.mp4 파일이 없습니다.")
else:
    print(f"🎬 총 {len(base_videos)}개 base.mp4에 대해 RIFE 보간 시작")

    success_count = 0

    for base_video in tqdm(base_videos, desc="🌀 RIFE 보간 중", unit="쌍"):
        pair_dir = os.path.dirname(base_video)
        pair_name = os.path.basename(base_video).replace("_base.mp4", "")
        rife_raw = os.path.join(pair_dir, f"{pair_name}_rife_raw.mp4")
        rife_final = os.path.join(pair_dir, f"{pair_name}_rife.mp4")

        if os.path.exists(rife_final):
            print(f"✅ {pair_name}: 이미 보간됨, 건너뜀")
            success_count += 1
            continue

        ok, out, err = interpolate_video_rife(base_video, rife_raw, TARGET_FPS)

        if ok and os.path.exists(rife_raw):
            # 재생 시간 맞춰 속도 조정
            if match_video_duration(base_video, rife_raw, rife_final):
                os.remove(rife_raw)
                print(f"✅ {pair_name}: 보간 + 시간조정 완료")
                success_count += 1
            else:
                print(f"❌ {pair_name}: 시간 조정 실패")
        else:
            print(f"❌ {pair_name}: 보간 실패\n{err}")

    print(f"\n🎉 완료: {success_count}/{len(base_videos)}개 보간 성공")

🎬 총 3개 base.mp4에 대해 RIFE 보간 시작


🌀 RIFE 보간 중:  33%|███▎      | 1/3 [00:06<00:13,  6.90s/쌍]

✅ pair_554_0554: 보간 + 시간조정 완료


🌀 RIFE 보간 중:  67%|██████▋   | 2/3 [00:13<00:06,  6.76s/쌍]

✅ pair_555_0555: 보간 + 시간조정 완료


🌀 RIFE 보간 중: 100%|██████████| 3/3 [00:20<00:00,  6.98s/쌍]

✅ pair_556_0556: 보간 + 시간조정 완료

🎉 완료: 3/3개 보간 성공


## 요건 기존 파이프 라인

In [ ]:
import cv2
import subprocess
import os
from glob import glob
import sys
import warnings
import re
from pathlib import Path
from tqdm import tqdm

# 로그 정리
warnings.filterwarnings('ignore')
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
os.environ['PYTHONWARNINGS'] = 'ignore'

class SimSwapAutoPipeline:
    def __init__(self):
        self.setup_simswap()

    def setup_simswap(self):
        """SimSwap 환경 설정"""
        print("🔧 SimSwap 환경 설정 중...")

        # SimSwap 디렉토리로 이동
        if not os.path.exists('/content/SimSwap'):
            print("❌ SimSwap이 설치되지 않았습니다. 먼저 SimSwap을 설치해주세요.")
            return False

        os.chdir('/content/SimSwap')

        # 필요한 경로들
        self.arcface_path = '/content/arcface_model/arcface_checkpoint.tar'
        self.checkpoints_dir = '/content/SimSwap/checkpoints'

        print("✅ SimSwap 환경 설정 완료")
        return True

    def extract_number_from_filename(self, filename):
        """파일명에서 숫자 추출 (영상과 이미지 매칭용)"""
        numbers = re.findall(r'\d+', os.path.splitext(filename)[0])
        if numbers:
            return int(numbers[0])
        return None

    def get_video_image_pairs(self, video_dir, image_dir):
        """영상과 이미지 파일들을 번호로 매칭"""
        video_files = glob(os.path.join(video_dir, "*.mp4"))
        image_files = glob(os.path.join(image_dir, "*.jpg")) + glob(os.path.join(image_dir, "*.png"))

        # 파일명에서 번호 추출하여 딕셔너리 생성
        video_dict = {}
        for video_path in video_files:
            filename = os.path.basename(video_path)
            number = self.extract_number_from_filename(filename)
            if number is not None:
                video_dict[number] = video_path

        image_dict = {}
        for image_path in image_files:
            filename = os.path.basename(image_path)
            number = self.extract_number_from_filename(filename)
            if number is not None:
                image_dict[number] = image_path

        # 매칭되는 쌍 찾기
        pairs = []
        for number in sorted(set(video_dict.keys()) & set(image_dict.keys())):
            pairs.append({
                'number': number,
                'video_path': video_dict[number],
                'image_path': image_dict[number],
                'video_name': os.path.splitext(os.path.basename(video_dict[number]))[0],
                'image_name': os.path.splitext(os.path.basename(image_dict[number]))[0]
            })

        return pairs

    def extract_frames(self, video_path, frame_dir, interval=5):
        """비디오에서 프레임 추출"""
        cap = cv2.VideoCapture(video_path)
        if not cap.isOpened():
            return None, None

        original_fps = cap.get(cv2.CAP_PROP_FPS)
        total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
        new_fps = original_fps / interval

        os.makedirs(frame_dir, exist_ok=True)

        frame_count = 0
        saved_count = 0

        with tqdm(total=total_frames, desc="  프레임 추출") as pbar:
            while True:
                ret, frame = cap.read()
                if not ret:
                    break

                if frame_count % interval == 0:
                    cv2.imwrite(os.path.join(frame_dir, f'frame_{saved_count:04d}.jpg'), frame)
                    saved_count += 1

                frame_count += 1
                pbar.update(1)

        cap.release()
        return original_fps, new_fps


    def run_simswap_on_frames(self, source_img, frame_dir, swap_dir):
        """SimSwap으로 프레임들 처리 - 에러 체크 추가"""
        frames = sorted([f for f in os.listdir(frame_dir) if f.endswith('.jpg')])
        total_frames = len(frames)

        os.makedirs(swap_dir, exist_ok=True)

        print(f"     소스 이미지: {source_img}")
        print(f"     소스 이미지 존재: {os.path.exists(source_img)}")

        success_count = 0

        with tqdm(total=total_frames, desc="  SimSwap 처리") as pbar:
            for i, frame_name in enumerate(frames):
                target_img = os.path.join(frame_dir, frame_name)
                result_subdir = os.path.join(swap_dir, f'frame_{i:04d}')
                os.makedirs(result_subdir, exist_ok=True)

                # SimSwap 실행 (에러 체크 포함)
                cmd = [
                    'python', 'test_wholeimage_swapsingle.py',
                    '--crop_size', '224',
                    '--use_mask',
                    '--no_simswaplogo',
                    '--name', 'people',
                    '--Arc_path', self.arcface_path,
                    '--pic_a_path', source_img,
                    '--pic_b_path', target_img,
                    '--pic_specific_path', target_img,
                    '--output_path', result_subdir,
                    '--checkpoints_dir', self.checkpoints_dir
                ]

                # 첫 번째 프레임은 에러 체크
                if i == 0:
                    print(f"     첫 번째 프레임 테스트 중...")
                    result = subprocess.run(cmd, capture_output=True, text=True)
                    if result.returncode != 0:
                        print(f"     ❌ SimSwap 에러:")
                        print(f"     stderr: {result.stderr[:500]}")
                        print(f"     stdout: {result.stdout[:500]}")
                    else:
                        print(f"     ✅ 첫 번째 프레임 성공")

                    # 결과 파일 확인
                    result_files = [f for f in os.listdir(result_subdir) if f.endswith(('.jpg', '.png'))]
                    print(f"     생성된 파일: {result_files}")

                    if result_files:
                        success_count += 1

                    # 나머지는 조용히 처리
                    pbar.update(1)
                    continue

                # 나머지 프레임들
                subprocess.run(cmd, stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL)

                # 결과 파일 확인
                result_files = [f for f in os.listdir(result_subdir) if f.endswith(('.jpg', '.png'))]
                if result_files:
                    success_count += 1

                pbar.update(1)

        print(f"     성공한 프레임: {success_count}/{total_frames}")
        return success_count > 0





    def create_video_from_swapped_results(self, swap_dir, output_video, target_fps):
        """SimSwap 결과를 비디오로 변환"""
        print(f"     swap_dir 확인: {swap_dir}")

        if not os.path.exists(swap_dir):
            print(f"     ❌ swap_dir이 존재하지 않음: {swap_dir}")
            return False

        result_images = []
        swap_subdirs = sorted([d for d in os.listdir(swap_dir) if os.path.isdir(os.path.join(swap_dir, d))])
        print(f"     발견된 하위 디렉토리: {len(swap_subdirs)}개")

        for subdir in swap_subdirs:
            subdir_path = os.path.join(swap_dir, subdir)
            result_files = [f for f in os.listdir(subdir_path) if f.endswith(('.jpg', '.png'))]
            print(f"     {subdir}: {len(result_files)}개 파일")
            if result_files:
                result_images.append(os.path.join(subdir_path, result_files[0]))

        print(f"     총 수집된 이미지: {len(result_images)}개")

        if not result_images:
            print("     ❌ 결과 이미지가 없음")
            return False

        first_img = cv2.imread(result_images[0])
        if first_img is None:
            print(f"     ❌ 첫 번째 이미지를 읽을 수 없음: {result_images[0]}")
            return False

        height, width, _ = first_img.shape
        print(f"     비디오 크기: {width}x{height}, FPS: {target_fps}")

        fourcc = cv2.VideoWriter_fourcc(*'mp4v')
        out = cv2.VideoWriter(output_video, fourcc, target_fps, (width, height))

        if not out.isOpened():
            print(f"     ❌ 비디오 라이터를 열 수 없음: {output_video}")
            return False

        print(f"     비디오 생성 시작...")
        with tqdm(total=len(result_images), desc="  비디오 생성") as pbar:
            for img_path in result_images:
                img = cv2.imread(img_path)
                if img is not None:
                    out.write(img)
                else:
                    print(f"     ⚠️ 이미지 읽기 실패: {img_path}")
                pbar.update(1)

        out.release()
        print(f"     ✅ 비디오 생성 완료: {output_video}")
        return True

    def interpolate_video(self, input_video, output_video, target_fps):
        """FFmpeg를 사용한 프레임 보간"""
        print(f"  보간 시작: {target_fps:.1f}FPS로 변환 중...")

        cmd = [
            'ffmpeg', '-i', input_video,
            '-filter:v', f'minterpolate=fps={target_fps}:mi_mode=mci:mc_mode=aobmc:vsbmc=1',
            '-c:v', 'libx264', '-crf', '23', '-preset', 'medium',
            '-y', output_video
        ]

        try:
            result = subprocess.run(cmd, capture_output=True, text=True)
            if result.returncode == 0:
                print(f"  ✅ 보간 완료: {output_video}")
                return True
            else:
                print(f"  ❌ 보간 실패")
                return False
        except Exception as e:
            print(f"  ❌ 보간 중 오류: {e}")
            return False

    def process_single_pair(self, video_path, source_image_path, output_dir, frame_interval=5):
        """단일 영상-이미지 쌍 처리"""
        # 디렉토리 설정
        frame_dir = os.path.join(output_dir, "extracted_frames")
        swap_dir = os.path.join(output_dir, "swapped_frames")

        # 1. 프레임 추출
        print("  1. 프레임 추출 중...")
        original_fps, new_fps = self.extract_frames(video_path, frame_dir, frame_interval)
        if original_fps is None:
            return False

        print(f"     원본 FPS: {original_fps:.1f} → 새 FPS: {new_fps:.1f}")

        # 2. SimSwap 처리
        print("  2. SimSwap 얼굴 교체 중...")
        self.run_simswap_on_frames(source_image_path, frame_dir, swap_dir)

        # 3. 기본 비디오 생성
        print("  3. 기본 비디오 생성 중...")
        base_video_path = os.path.join(output_dir, "base_video.mp4")
        if not self.create_video_from_swapped_results(swap_dir, base_video_path, new_fps):
            return False

        return True, original_fps, new_fps, base_video_path

    def process_all_pairs(self, video_dir="/content/input_videos",
                         image_dir="/content/experiments/restyle_e4e_sg3/inference_results/0",
                         output_base_dir="/content/simswap_results",
                         frame_interval=5, use_interpolation=True):
        """모든 영상-이미지 쌍 자동 처리"""

        # 매칭되는 쌍 찾기
        pairs = self.get_video_image_pairs(video_dir, image_dir)

        if not pairs:
            print("❌ 매칭되는 영상-이미지 쌍을 찾을 수 없습니다!")
            print(f"영상 디렉토리: {video_dir}")
            print(f"이미지 디렉토리: {image_dir}")
            return

        print(f"🎯 총 {len(pairs)}개의 매칭 쌍을 찾았습니다!")

        # 기본 출력 디렉토리 생성
        os.makedirs(output_base_dir, exist_ok=True)

        success_count = 0

        for i, pair in enumerate(pairs, 1):
            print(f"\n[{i}/{len(pairs)}] 처리 중...")
            print(f"  영상: {os.path.basename(pair['video_path'])}")
            print(f"  이미지: {os.path.basename(pair['image_path'])}")

            # 개별 출력 디렉토리 생성
            pair_name = f"pair_{pair['number']:03d}_{pair['video_name']}"
            pair_output_dir = os.path.join(output_base_dir, pair_name)

            try:
                # 1. SimSwap 처리
                result = self.process_single_pair(
                    pair['video_path'],
                    pair['image_path'],
                    pair_output_dir,
                    frame_interval
                )

                if not result:
                    print(f"  ❌ 쌍 {pair['number']} 처리 실패")
                    continue

                success, original_fps, base_fps, base_video_path = result

                # 2. 보간된 비디오 생성 (선택사항)
                if use_interpolation:
                    final_video_path = os.path.join(pair_output_dir, f"{pair_name}_final.mp4")
                    print("  4. 프레임 보간 중...")
                    if self.interpolate_video(base_video_path, final_video_path, original_fps):
                        print(f"  🎉 최종 결과: {final_video_path}")
                    else:
                        print(f"  ⚠️ 보간 실패, 기본 비디오 사용: {base_video_path}")
                else:
                    print(f"  🎉 최종 결과: {base_video_path}")

                success_count += 1

            except Exception as e:
                print(f"  ❌ 쌍 {pair['number']} 처리 중 오류: {e}")
                continue

        print(f"\n🎉 처리 완료! {success_count}/{len(pairs)}개 성공")
        print(f"결과 저장 위치: {output_base_dir}")

from multiprocessing import Pool

def process_single_pair_wrapper(args):
    pipeline, video_path, image_path, output_dir, frame_interval = args
    return pipeline.process_single_pair(video_path, image_path, output_dir, frame_interval)

def process_all_pairs_parallel(pipeline, pairs, output_base_dir, frame_interval=5, use_interpolation=True, max_workers=3):
    tasks = []
    for pair in pairs:
        pair_name = f"pair_{pair['number']:03d}_{pair['video_name']}"
        pair_output_dir = os.path.join(output_base_dir, pair_name)
        tasks.append((pipeline, pair['video_path'], pair['image_path'], pair_output_dir, frame_interval))
    with Pool(processes=max_workers) as pool:
        pool.map(process_single_pair_wrapper, tasks)

# 사용 예시
if __name__ == "__main__":
    # 파이프라인 초기화
    pipeline = SimSwapAutoPipeline()

    # 설정
    VIDEO_DIR = "/content/input_videos"
    IMAGE_DIR = "/content/experiments/restyle_e4e_sg3/inference_results/0"
    OUTPUT_DIR = "/content/simswap_results"
    FRAME_INTERVAL = 5  # 5프레임마다 샘플링
    USE_INTERPOLATION = True  # 프레임 보간 사용 여부

    print("=== SimSwap 자동 파이프라인 시작 ===")
    print(f"영상 디렉토리: {VIDEO_DIR}")
    print(f"이미지 디렉토리: {IMAGE_DIR}")
    print(f"출력 디렉토리: {OUTPUT_DIR}")
    print(f"프레임 간격: {FRAME_INTERVAL}")
    print(f"보간 사용: {'예' if USE_INTERPOLATION else '아니오'}")

    pairs = pipeline.get_video_image_pairs(VIDEO_DIR, IMAGE_DIR)

  process_all_pairs_parallel(
      pipeline, pairs,
      output_base_dir=OUTPUT_DIR,
      frame_interval=FRAME_INTERVAL,
      use_interpolation=USE_INTERPOLATION,
      max_workers=3  # 병렬 갯수 (T4 2장 쓰면 2~3, P100은 1~2 권장)
  )

In [ ]:
!rm -rf /content/simswap_results/*

In [ ]:
!pip list
